In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# 确保在 CPU 上运行，以便观察内存地址等细节
device = torch.device("cpu")

# --- 1. 定义模型和数据 ---
# 简化版两层 MLP
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleMLP, self).__init__()
        # fc1 层：输入 input_dim，输出 hidden_dim
        # nn.Linear 会自动创建 weight 和 bias
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        # 激活函数
        self.relu = nn.ReLU()
        # fc2 层：输入 hidden_dim，输出 output_dim
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x -> fc1 -> relu -> fc2 -> output
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 模拟数据
input_dim = 4      # 每张图片的特征数（比如 2x2 展平后）
hidden_dim = 5     # 隐藏层神经元数
output_dim = 10    # 10 个数字类别 (0-9)
batch_size = 5     # 5 张图片

# 模拟输入数据 X，不需要求导
X = torch.randn(batch_size, input_dim, requires_grad=False).to(device)
# 模拟标签 y
y_true = torch.randint(0, output_dim, (batch_size,), dtype=torch.long).to(device)

# 实例化模型
model = SimpleMLP(input_dim, hidden_dim, output_dim).to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

print("--- 初始状态检查 ---")
print(f"X (输入数据): shape={X.shape}, requires_grad={X.requires_grad}, grad_fn={X.grad_fn}")
print(f"y_true (标签): shape={y_true.shape}, requires_grad={y_true.requires_grad}, grad_fn={y_true.grad_fn}\n")

# 检查模型的参数 W1, b1, W2, b2
# model.named_parameters() 返回参数名称和参数张量本身
print("模型参数初始状态:")
for name, param in model.named_parameters():
    print(f"  {name}: shape={param.shape}, requires_grad={param.requires_grad}, grad_fn={param.grad_fn}, grad={param.grad}")
    # 注意：所有参数的 requires_grad 都是 True，因为它们需要被优化
    # grad_fn 都是 None，因为它们是“叶子节点”，是直接创建的，不是通过运算得来。
    # grad 都是 None，因为还没有进行反向传播。
print("-" * 50)


# --- 2. 前向传播：构建计算图 ---
print("\n--- 前向传播开始：一步步看 grad_fn 的变化 ---")

# 步骤 2.0: 清零梯度 (习惯上，虽然这里是第一次，但模型参数可能已经有历史梯度)
optimizer.zero_grad()
print("所有模型参数的 .grad 已清零。")

# 步骤 2.1: 通过 fc1 层
# x 是 (5, 4)， self.fc1.weight 是 (hidden_dim, input_dim) 即 (5, 4)
# 矩阵乘法 X @ W_transpose + b
# (5, 4) @ (4, 5) -> (5, 5) + (5,) -> (5, 5)
fc1_output = model.fc1(X)
print(f"\nfc1_output (X @ W1.T + b1): shape={fc1_output.shape}")
print(f"  requires_grad={fc1_output.requires_grad}") # True
# 此时 fc1_output 的 grad_fn 会是一个代表线性层操作的 Function
# (如 AddmmBackward0，因为它包含了矩阵乘法和加法)
print(f"  grad_fn={fc1_output.grad_fn}")
# fc1_output 内部的 grad_fn 会记住：
#   - 它是由 model.fc1 这个 nn.Linear 模块产生的。
#   - 它需要 model.fc1.weight 和 model.fc1.bias 来计算梯度。
#   - 它会存储 X（输入）的引用，以便计算 X 的梯度（如果 X 允许求导）。


# 步骤 2.2: 通过 ReLU 激活函数
relu_output = model.relu(fc1_output)
print(f"\nrelu_output (ReLU(fc1_output)): shape={relu_output.shape}")
print(f"  requires_grad={relu_output.requires_grad}") # True
# relu_output 的 grad_fn 会是一个代表 ReLU 操作的 Function
print(f"  grad_fn={relu_output.grad_fn}") # 例如 <ReLUBackward0 object at ...>
# relu_output 内部的 grad_fn 会记住：
#   - 它的输入是 fc1_output。
#   - 它会存储 fc1_output 的引用以及在 ReLU 操作中哪些值是正的（用于梯度计算）。


# 步骤 2.3: 通过 fc2 层 (最终输出 logits)
# relu_output 是 (5, 5)， self.fc2.weight 是 (output_dim, hidden_dim) 即 (10, 5)
# (5, 5) @ (5, 10) -> (5, 10) + (10,) -> (5, 10)
logits = model.fc2(relu_output)
print(f"\nlogits (最终输出): shape={logits.shape}")
print(f"  requires_grad={logits.requires_grad}") # True
# logits 的 grad_fn 会是一个代表线性层操作的 Function
print(f"  grad_fn={logits.grad_fn}") # 例如 <AddmmBackward0 object at ...>
# logits 内部的 grad_fn 会记住：
#   - 它是通过 model.fc2 产生的。
#   - 它需要 model.fc2.weight 和 model.fc2.bias 来计算梯度。
#   - 它会存储 relu_output 的引用。


# 步骤 2.4: 计算损失 (交叉熵)
# criterion(logits, y_true)
# CrossEntropyLoss 内部会先对 logits 进行 LogSoftmax，再进行 NLLLoss
loss = criterion(logits, y_true)
print(f"\nLoss (交叉熵损失): shape={loss.shape}")
print(f"  requires_grad={loss.requires_grad}") # True
# 损失张量通常是标量 (shape=torch.Size([])), 它的 grad_fn 会指向损失函数的反向计算
print(f"  grad_fn={loss.grad_fn}") # 例如 <NllLossBackward0 object at ...>
# loss 内部的 grad_fn 会记住：
#   - 它的输入是 logits 和 y_true。
#   - 它需要 logits 的值和 y_true 的值来计算梯度。

print("\n此时，一个从 loss 回溯到 W1, b1, W2, b2 的计算图已动态构建完成。")
print("-" * 50)


# --- 3. 反向传播：计算梯度 ---
print("\n--- 反向传播开始：从 Loss 回溯计算梯度 ---")

# 调用 backward() 方法
loss.backward()

print("\n反向传播完成！检查参数的 .grad 属性。")
# 检查模型参数的梯度
for name, param in model.named_parameters():
    print(f"  {name}:")
    print(f"    - grad_fn: {param.grad_fn}") # 仍是 None，因为它们是叶子节点
    print(f"    - grad is None: {param.grad is None}")
    if param.grad is not None:
        print(f"    - grad.shape: {param.grad.shape}")
        # 梯度值：是 loss 对该参数中每个元素的偏导数
        # 它们是和参数形状相同的 Tensor
        # print(f"    - grad value: \n{param.grad}") # 值比较大，不打印了，但你可以自行打印
print("-" * 50)


# --- 4. 参数更新：优化器根据梯度更新参数 ---
print("\n--- 参数更新阶段：优化器利用 .grad 更新参数 ---")

# 获取参数更新前的值 (只取一部分为例)
old_fc1_weight_val = model.fc1.weight.data[0, 0].item()
old_fc2_bias_val = model.fc2.bias.data[0].item()
print(f"更新前 model.fc1.weight[0,0]: {old_fc1_weight_val:.4f}")
print(f"更新前 model.fc2.bias[0]: {old_fc2_bias_val:.4f}")

optimizer.step() # 优化器执行一步更新

print("\n优化器更新完成！参数值已改变，.grad 已清零。")
# 检查参数更新后的值和梯度状态
for name, param in model.named_parameters():
    print(f"  {name}:")
    # 梯度已经被清零了 (通常是 None，或者一个全零张量，取决于优化器实现)
    print(f"    - grad is None: {param.grad is None}")

# 获取参数更新后的值
new_fc1_weight_val = model.fc1.weight.data[0, 0].item()
new_fc2_bias_val = model.fc2.bias.data[0].item()
print(f"更新后 model.fc1.weight[0,0]: {new_fc1_weight_val:.4f}")
print(f"更新后 model.fc2.bias[0]: {new_fc2_bias_val:.4f}")
print(f"fc1.weight[0,0] 变化量: {new_fc1_weight_val - old_fc1_weight_val:.4f}")
print(f"fc2.bias[0] 变化量: {new_fc2_bias_val - old_fc2_bias_val:.4f}")

# 可以重复整个循环（前向传播、反向传播、参数更新）来进行多轮训练
print("\n--- 一个完整的训练步骤完成 ---")

--- 初始状态检查 ---
X (输入数据): shape=torch.Size([5, 4]), requires_grad=False, grad_fn=None
y_true (标签): shape=torch.Size([5]), requires_grad=False, grad_fn=None

模型参数初始状态:
  fc1.weight: shape=torch.Size([5, 4]), requires_grad=True, grad_fn=None, grad=None
  fc1.bias: shape=torch.Size([5]), requires_grad=True, grad_fn=None, grad=None
  fc2.weight: shape=torch.Size([10, 5]), requires_grad=True, grad_fn=None, grad=None
  fc2.bias: shape=torch.Size([10]), requires_grad=True, grad_fn=None, grad=None
--------------------------------------------------

--- 前向传播开始：一步步看 grad_fn 的变化 ---
所有模型参数的 .grad 已清零。

fc1_output (X @ W1.T + b1): shape=torch.Size([5, 5])
  requires_grad=True
  grad_fn=<AddmmBackward0 object at 0x7816058016f0>

relu_output (ReLU(fc1_output)): shape=torch.Size([5, 5])
  requires_grad=True
  grad_fn=<ReluBackward0 object at 0x7816055b14e0>

logits (最终输出): shape=torch.Size([5, 10])
  requires_grad=True
  grad_fn=<AddmmBackward0 object at 0x7816058016f0>

Loss (交叉熵损失): shape=torch.Size